Develop two models using only the real estate transaction data:
a. One to estimate property sale prices.
b. Another to estimate property rental prices. (15 points)

Create two additional models incorporating both real estate data and supplementary datasets:
a. One to estimate property sale prices.
b. Another to estimate property rental prices. (15 points)

Evaluate the performance of all four models using metrics such as RMSE, MSE, and R². Analyze whether the inclusion of additional data enhances model accuracy. (10 points)

You should at least train your model on these 9 features but you are free to add more:
Property Type or/and Property Sub Type
Number of rooms/bedrooms
Property Size (sq.m)
Area/Neighborhood
Nearest Metro
Nearest Mall
Nearest Landmark
Usage (commercial/residential)
Average prices the previous month/week (for the same kind of property)